In [1]:
import json

with open('malay-asr-test.json') as fopen:
    data = json.load(fopen)
    
X, Y = data['X'], data['Y']

In [11]:
import malaya_speech
import speech_recognition as sr

r = sr.Recognizer()

In [3]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [5]:
X[0]

'/home/husein/ssd1/speech-bahasa/ms_my/audio/test/6642689329059010818.wav'

In [15]:
from tqdm import tqdm
import soundfile as sf

wer, cer = [], []
for i in tqdm(range(len(X))):
    
    try:
        y, sr_ = sf.read(X[i])
        sf.write('test.wav', y, sr_)
        with sr.AudioFile('test.wav') as source:
            a = r.record(source)

        text = r.recognize_google(a, language = 'ms')
        text = preprocessing_text(text)
        
        wer.append(malaya_speech.metrics.calculate_wer(Y[i], text))
        cer.append(malaya_speech.metrics.calculate_cer(Y[i], text))
    except Exception as e:
        print(e)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 739/739 [29:40<00:00,  2.41s/it]


In [14]:
import numpy as np

np.mean(wer), np.mean(cer)

(0.10958877995642702, 0.04789152717751709)